In [1]:

import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import tensorflow.keras as keras

2024-01-16 23:14:06.636709: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-16 23:14:06.657613: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-16 23:14:06.657634: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-16 23:14:06.658142: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-16 23:14:06.661556: I tensorflow/core/platform/cpu_feature_guar

In [2]:
x = pd.read_csv("Cary_Weather_Data.csv")
x['DATE'] = pd.to_datetime(x[['YEAR', 'MONTH', 'DAY']])
x = x.drop(columns=['YEAR','MONTH','DAY'])
#x = x.drop(columns=['Humidity','Precipitation','Wind Speed'])
x = x[['DATE', 'Temperature', 'Humidity', 'Precipitation', 'Wind Speed']]
x.head()
x.set_index('DATE', inplace=True)
x.head()


,Temperature,Humidity,Precipitation,Wind Speed
DATE,,,,
1981-07-01,15.12,10.68,6.65,3.59
1981-07-02,7.53,13.12,10.68,1.64
1981-07-03,7.32,13.79,37.42,2.43
1981-07-04,10.43,14.40,8.41,2.29
1981-07-05,13.13,15.20,4.13,1.58


In [3]:
scaler = StandardScaler()
x_scale = scaler.fit_transform(x)
x_scale = pd.DataFrame(x_scale, columns=x.columns, index=x.index)
x_scale = x_scale.drop(columns=['Humidity', 'Precipitation', 'Wind Speed'])
x_scale.head()

,Temperature
DATE,
1981-07-01,0.926990
1981-07-02,-1.305674
1981-07-03,-1.367448
1981-07-04,-0.452614
1981-07-05,0.341614


In [4]:
def create_dataset(X, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X.iloc[i:(i + time_steps)].values)
        ys.append(X.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

In [5]:
time_steps = 1

X, y = create_dataset(x_scale, time_steps)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=123, shuffle=False)
X_train.shape
X_val.shape

(3106, 1, 1)

In [6]:
model = tf.keras.models.Sequential([
    # LSTM Layer 1 with return_sequences=True
    keras.layers.LSTM(256, input_shape=(time_steps, 1), return_sequences=True, name='lstm_layer_1'),
    keras.layers.Dropout(0.2, name='dropout_layer_1'),
    
    # LSTM Layer 2 with return_sequences=True
    keras.layers.LSTM(128, return_sequences=True, name='lstm_layer_2'),
    keras.layers.Dropout(0.2, name='dropout_layer_2'),
    
    # Bidirectional LSTM Layer with return_sequences=True
    keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True), name='bidirectional_lstm_layer'),
    keras.layers.Dropout(0.2, name='dropout_layer_3'),
    
    # Attention Layer
    #keras.layers.Attention(name='attention_layer'),
    
    # Global Max Pooling 1D Layer
    keras.layers.GlobalMaxPooling1D(name='global_max_pooling_layer'),
    
    # Dense Layer 1
    keras.layers.Dense(128, activation='relu', name='dense_layer_1'),
    keras.layers.Dropout(0.3, name='dropout_layer_4'),
    
    # Dense Layer 2
    keras.layers.Dense(64, activation='relu', name='dense_layer_2'),
    
    # Output Layer with Linear Activation
    keras.layers.Dense(1, activation='linear', name='output_layer')
])



cp = tf.keras.callbacks.ModelCheckpoint('model', save_best_only=True)
model.summary()

2024-01-16 23:14:08.467108: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-16 23:14:08.496964: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-16 23:14:08.506187: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_layer_1 (LSTM)         (None, 1, 256)            264192    
                                                                 
 dropout_layer_1 (Dropout)   (None, 1, 256)            0         
                                                                 
 lstm_layer_2 (LSTM)         (None, 1, 128)            197120    
                                                                 
 dropout_layer_2 (Dropout)   (None, 1, 128)            0         
                                                                 
 bidirectional_lstm_layer (  (None, 1, 128)            98816     
 Bidirectional)                                                  
                                                                 
 dropout_layer_3 (Dropout)   (None, 1, 128)            0         
                                                        

In [7]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

# Compile the model with Mean Squared Error (MSE) loss and Mean Absolute Error (MAE) metric
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])


In [8]:
cp = tf.keras.callbacks.ModelCheckpoint('best_model.keras', save_best_only=True)
history = model.fit(X_train, y_train, epochs=10, callbacks=cp)

Epoch 1/10


2024-01-16 23:14:11.216627: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-01-16 23:14:11.757501: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fc6b553bd10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-16 23:14:11.757515: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2024-01-16 23:14:11.760319: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1705464851.803268   20734 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


389/389 [==============================] - 6s 9ms/step - loss: 0.8960 - mae: 0.7513
Epoch 2/10
389/389 [==============================] - 4s 10ms/step - loss: 0.8375 - mae: 0.7249
Epoch 3/10
389/389 [==============================] - 4s 10ms/step - loss: 0.8375 - mae: 0.7246
Epoch 4/10
389/389 [==============================] - 4s 9ms/step - loss: 0.8366 - mae: 0.7250
Epoch 5/10
389/389 [==============================] - 3s 9ms/step - loss: 0.8356 - mae: 0.7242
Epoch 6/10
389/389 [==============================] - 4s 10ms/step - loss: 0.8361 - mae: 0.7248
Epoch 7/10
389/389 [==============================] - 4s 10ms/step - loss: 0.8345 - mae: 0.7243
Epoch 8/10
389/389 [==============================] - 3s 8ms/step - loss: 0.8355 - mae: 0.7242
Epoch 9/10
389/389 [==============================] - 3s 8ms/step - loss: 0.8351 - mae: 0.7238
Epoch 10/10
389/389 [==============================] - 4s 10ms/step - loss: 0.8357 - mae: 0.7239


In [11]:
matplotlib.interactive(True)
%matplotlib qt
plt.plot(history.history['loss'], label='Training Loss')
#plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

 1/98 [..............................] - ETA: 57s

In [12]:
prediction = model.predict(X_val)
prediction_reshape = prediction[:, 0]
x_val_reshz = X_val[:, 0, 0]

98/98 [==============================] - 0s 1ms/step


In [13]:
time_val = x.index[-len(X_val):]

In [14]:
plt.plot(x_scale['Temperature'], color='green')
plt.plot(time_val, x_val_reshz, label='Actual', color='blue')
plt.plot(time_val, prediction_reshape, label='Prediction', color='orange')
plt.show()